In [ ]:
from utils import petab as pet
import numpy as np
import utils.sbml as sbml
import utils.amici as am
from models.sbml_models import CRP2_CPE
%load_ext autoreload
%autoreload 2

### Define SBML/AMICI Model

In [ ]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',  
}, noise_value=0.02)

# Load SBML/AMICI model
sbml_model_filepath = sbml.write_model(CRP2_CPE)

# validator = sbml.validateSBML(ucheck=False)
# validator.validate(sbml_model_filepath)
# model_name, model_output_dir = am.compile_amici_model(
#     sbml_model_filepath, observables_df, verbose=False
# )
model_name = 'CRP2_CPE'
model_output_dir = '/SBML/amici_models/CRP2_CPE/'
model = am.load_amici_model(model_name, model_output_dir)


### Define Measurements

In [ ]:
# Define model parameters
true_params = {
    # 'kpAA': 1.0,
    "rA": 5.0,
    "rB": 1.0,
    "rX": 1.0,
    "KAA": 0.0,
    "KAB": 0.0,
    "KBA": 0.0,
    "KBB": 0.0,
}
model = am.set_model_parameters(model, true_params)

# Define conditions
fA = np.array([0.1, 0.25, 0.5, 0.75, 0.9], dtype=float)
cM = np.array([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float)
A0 = fA * cM
B0 = (1 - fA) * cM

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        "A0": list(A0),
        "B0": list(B0),
    }
)

# Define conversion values
timepoints = list(np.arange(0, 1, 0.1, dtype=float))
# print(timepoints)

measurements_df, rdatas = pet.define_measurements(
    amici_model=model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True,
)

measurements_df

In [ ]:
from models.cpe_models import CopolymerEquations as CPE

cpe = CPE.from_ratios(true_params.values(), fA0=0.5, M0=1.0)
cpe.t_eval = timepoints
cpe.solve_izu()